In [ ]:
from datetime import date

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, lit, round, to_date, when
from pyspark.sql.window import Window

spark = SparkSession.builder \
    .appName("edwin_app") \
    .config("spark.some.config.option", "valor") \
    .getOrCreate()

In [41]:
# Fecha a consultar particion

load_date = date.today()
print(load_date)

2025-05-27


In [42]:
df = spark.read.parquet('./outputs/edwin/raw_stage/all_policies_table')

In [43]:
# Filtrar unicamente clientes Empleados

df_1 = df.filter((col('EmploymentStatus') == lit('Employed'))&
                 (col('load_date') == lit(load_date))
                 )

In [44]:
df_1.count()

5698

In [45]:
# Casteo de columnas numericas

df_1a = df_1.select('*',
                   col('Income').cast('int').alias('income_by_year'),
                   round(col('Customer Lifetime Value'), 2).alias('customer_lifetime_value').cast('float')
                   ).drop('Income',
                           'Customer Lifetime Value')

In [46]:
# Crear columna ingreso por mes y renombrar Income por Income_by_year

df_2 = df_1a.select('*',
               round((col('income_by_year') / lit(12)), 2).alias('income_by_month')
               ).drop(col('income_by_year'))

In [47]:
# Calcular porcentaje del ingreso mensual destinado al pago de la poliza

df_2a = df_2.select('*',
                   round(((col('Monthly Premium Auto') / col('income_by_month'))) * 100, 2).alias('premium_income_ratio')
                   )

In [48]:
# Calcular la tasa de quejas de cada cliente por mes

df_2b = df_2a.select('*',
                     round(((col('Number of Open Complaints') / col('Months Since Policy Inception'))) * 100, 2).alias('complaint_rate')
                     )

In [49]:
df_2b.show(5, False)

+--------+--------+--------+---------+-----------------+----------------+------+-------------+--------------+--------------------+-----------------------+-----------------------------+-------------------------+------------------+----------------+-------------+------------------+-------------+------------+--------------+------------+----------+----------+-----------------------+---------------+--------------------+--------------+
|Customer|Response|Coverage|Education|Effective To Date|EmploymentStatus|Gender|Location Code|Marital Status|Monthly Premium Auto|Months Since Last Claim|Months Since Policy Inception|Number of Open Complaints|Number of Policies|Renew Offer Type|Sales Channel|Total Claim Amount|Vehicle Class|Vehicle Size|Policy Type   |Policy      |State     |load_date |customer_lifetime_value|income_by_month|premium_income_ratio|complaint_rate|
+--------+--------+--------+---------+-----------------+----------------+------+-------------+--------------+--------------------+----

In [50]:
# Categorizar Customer Lifetime Value como Alto Medio y Bajo

df_3 = df_2b.select('*',
                   when(col('customer_lifetime_value') <= 4000, 'BAJO')
                   .when((col('customer_lifetime_value') >= 4001) & (col('customer_lifetime_value') <=6000), 'MEDIO')
                   .when(col('customer_lifetime_value') >= 6001, 'ALTO')
                   .otherwise('DESCONOCIDO').alias('clv_categorical')
                        )

In [51]:
df_3.show(5, False)

+--------+--------+--------+---------+-----------------+----------------+------+-------------+--------------+--------------------+-----------------------+-----------------------------+-------------------------+------------------+----------------+-------------+------------------+-------------+------------+--------------+------------+----------+----------+-----------------------+---------------+--------------------+--------------+---------------+
|Customer|Response|Coverage|Education|Effective To Date|EmploymentStatus|Gender|Location Code|Marital Status|Monthly Premium Auto|Months Since Last Claim|Months Since Policy Inception|Number of Open Complaints|Number of Policies|Renew Offer Type|Sales Channel|Total Claim Amount|Vehicle Class|Vehicle Size|Policy Type   |Policy      |State     |load_date |customer_lifetime_value|income_by_month|premium_income_ratio|complaint_rate|clv_categorical|
+--------+--------+--------+---------+-----------------+----------------+------+-------------+--------

In [52]:
# Porcentaje que corresponde a cada categoria clv_categorial

w = Window.partitionBy(col('clv_categorical'))
total = df_3.count()

df_4 = df_3.select('*',
            count(col('clv_categorical')).over(w).alias('clv_cat_count'),
            )

df_4 = df_4.select('*',
                   round(((col('clv_cat_count') / lit(total))* 100), 1).alias('clv_categorical_percent')
                   ).drop('clv_cat_count')

In [53]:
df_5 = df_4.select('*',
                   lit(load_date).alias('load_date')
                   )

In [55]:
print(df_5.show(10, False))
print(df_5.count())


+--------+--------+--------+--------------------+-----------------+----------------+------+-------------+--------------+--------------------+-----------------------+-----------------------------+-------------------------+------------------+----------------+-------------+------------------+-------------+------------+--------------+------------+----------+----------+-----------------------+---------------+--------------------+--------------+---------------+-----------------------+----------+
|Customer|Response|Coverage|Education           |Effective To Date|EmploymentStatus|Gender|Location Code|Marital Status|Monthly Premium Auto|Months Since Last Claim|Months Since Policy Inception|Number of Open Complaints|Number of Policies|Renew Offer Type|Sales Channel|Total Claim Amount|Vehicle Class|Vehicle Size|Policy Type   |Policy      |State     |load_date |customer_lifetime_value|income_by_month|premium_income_ratio|complaint_rate|clv_categorical|clv_categorical_percent|load_date |
+--------+

In [60]:
df_6 = df_5.withColumn("Effective To Date", to_date("Effective To Date", "M/d/yy"))

In [61]:
df_6.printSchema()

root
 |-- Customer: string (nullable = true)
 |-- Response: string (nullable = true)
 |-- Coverage: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Effective To Date: date (nullable = true)
 |-- EmploymentStatus: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Location Code: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Monthly Premium Auto: string (nullable = true)
 |-- Months Since Last Claim: string (nullable = true)
 |-- Months Since Policy Inception: string (nullable = true)
 |-- Number of Open Complaints: string (nullable = true)
 |-- Number of Policies: string (nullable = true)
 |-- Renew Offer Type: string (nullable = true)
 |-- Sales Channel: string (nullable = true)
 |-- Total Claim Amount: string (nullable = true)
 |-- Vehicle Class: string (nullable = true)
 |-- Vehicle Size: string (nullable = true)
 |-- Policy Type: string (nullable = true)
 |-- Policy: string (nullable = true)
 |-- State: string (nu